In [1]:
import pandas as pd
import re

In [2]:
#read data
data = pd.read_csv("horses.csv")
data

,_key,id,lang,hits,name,thumbnail,thumbnail_width,thumbnail_height,catid,slug,...,attr/ct91/title/0,attr/ct91/type,attr/ct88/value/1,attr/ct88/value/2,attr/ct88/frontend_value/1,attr/ct88/frontend_value/2,attr/ct12/value/1,attr/ct12/frontend_value/1,attr/ct5/value/2,attr/ct5/frontend_value/2
0,item_3552,3552,en-GB,47,Perlino Turkoman,igallery/7401-7500/resized/RDR2_Horses_Turkoma...,320,180,34,3552:perlino-turkoman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,item_3551,3551,en-GB,53,Chestnut Turkoman,igallery/7401-7500/resized/RDR2_Horses_Turkoma...,320,180,34,3551:chestnut-turkoman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_3550,3550,en-GB,55,Black Turkoman,igallery/7401-7500/resized/RDR2_Horses_Turkoma...,320,180,34,3550:black-turkoman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,item_3549,3549,en-GB,74,Grey Turkoman,igallery/7401-7500/resized/RDR2_Horses_Turkoma...,320,180,34,3549:grey-turkoman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,item_3044,3044,en-GB,358,Red Dun Overo Mustang,igallery/7201-7300/resized/RDR2_Mustang_RedDun...,320,180,34,3044:red-dun-overo-mustang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,item_1491,1491,en-GB,28372,Mahogany Bay Tennessee Walker,igallery/2601-2700/resized/RDR2_Horses_Tenness...,320,180,34,1491:mahogany-bay-tennessee-walker,...,Story: Available after,multiple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,item_1346,1346,en-GB,44705,Iron Grey Ardennes (War Horse),images/red-dead-redemption-2/horses/resized/wa...,320,180,34,1346:iron-grey-ardennes,...,Story: Available after,multiple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,item_1343,1343,en-GB,96556,Red Chestnut Arabian Horse,igallery/2501-2600/resized/RDR2_EarlyAccess_Re...,320,180,34,1343:red-chestnut-arabian-horse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,item_1338,1338,en-GB,60285,Dappled Black Thoroughbred,images/red-dead-redemption-2/horses/resized/da...,320,180,34,1338:dappled-black-thoroughbred,...,Story: Available after,multiple,NaN,NaN,NaN,NaN,NaN,NaN,special-edition,Special Edition


The columns in this dataframe have a particular shape. The values displayed on the webpage are stored in the "attr/ct##/frontend_value/0" (where # is an integer) columns while the actual titles of said columns are in "attr/ct##/title/0" columns

In [3]:
#create a dictionary with the values column title as key and the actual title as value
columns_dictionary =  {c: c for c in data.columns }
for key, _ in columns_dictionary.items():
    if 'frontend_value/0' in key:
        title_column = key.replace('frontend_value', 'title')
        title_string = data[title_column].dropna().unique()[0]
        columns_dictionary[key] = title_string 
#change 'thumbnail' column title to 'image'
columns_dictionary['thumbnail'] = 'image'
columns_dictionary 

{'_key': '_key',
 'id': 'id',
 'lang': 'lang',
 'hits': 'hits',
 'name': 'name',
 'thumbnail': 'image',
 'thumbnail_width': 'thumbnail_width',
 'thumbnail_height': 'thumbnail_height',
 'catid': 'catid',
 'slug': 'slug',
 'catslug': 'catslug',
 'attr/ct90/value/0': 'attr/ct90/value/0',
 'attr/ct90/frontend_value/0': 'Horse Breed',
 'attr/ct90/params/render_class': 'attr/ct90/params/render_class',
 'attr/ct90/params/label_render_class': 'attr/ct90/params/label_render_class',
 'attr/ct90/title/0': 'attr/ct90/title/0',
 'attr/ct90/type': 'attr/ct90/type',
 'attr/ct88/value/0': 'attr/ct88/value/0',
 'attr/ct88/frontend_value/0': 'Horse Class',
 'attr/ct88/params/render_class': 'attr/ct88/params/render_class',
 'attr/ct88/params/label_render_class': 'attr/ct88/params/label_render_class',
 'attr/ct88/title/0': 'attr/ct88/title/0',
 'attr/ct88/type': 'attr/ct88/type',
 'attr/ct89/value/0': 'attr/ct89/value/0',
 'attr/ct89/frontend_value/0': 'Handling',
 'attr/ct89/params/render_class': 'attr/c

In [4]:
#rename columns
correct_column_data = data.rename(columns = columns_dictionary)

In [5]:
#select desired columns
reduced_data = correct_column_data[["id", "name", "image", "Horse Breed", "Horse Class", "Handling", "Health", "Stamina", "Speed", "Acceleration", "Handling", "Release Date", "Title Update", "Game Edition", "Overall Rating", "Base Value ($)"]]

In [6]:
#create deep copy of reduced_data df
final_data = reduced_data.copy()
#add full url to image column
final_data["image"] = 'https://www.gtabase.com/' + final_data["image"].astype(str)
#convert epoch to date time and format to YYYY-MM-DD
final_data["Release Date"] = pd.to_datetime(final_data["Release Date"], unit='s').dt.strftime('%Y-%m-%d')
#add one day offset to release date
final_data["Release Date"] = pd.DatetimeIndex(final_data["Release Date"]) + pd.DateOffset(1)

In [7]:
output = final_data.to_csv("output.csv")